# D(St)reams of Anomalies

a project for EECS 731 by Benjamin Wyss

Examining New York City taxi passenger quantities to detect anomalies in the total number of passengers per thirty minute interval.

###### python imports

In [5]:
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
plt.close('all')
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

### Reading Data Set From CSV

Numenta Anomaly Benchmark (NAB) Real Known Cause NYC Taxi Dataset

Taken from: https://github.com/numenta/NAB/tree/master/data on 10/15/20

###### Historical Product Demand Data Set

In [6]:
df = pd.read_csv('../data/raw/nyc_taxi.csv')

In [7]:
df

,timestamp,value
0,2014-07-01 00:00:00,10844
1,2014-07-01 00:30:00,8127
2,2014-07-01 01:00:00,6210
3,2014-07-01 01:30:00,4656
4,2014-07-01 02:00:00,3820
...,...,...
10315,2015-01-31 21:30:00,24670
10316,2015-01-31 22:00:00,25721
10317,2015-01-31 22:30:00,27309
10318,2015-01-31 23:00:00,26591


## The Big Ideas

Feature engineering and transformation can add value to this data set for constructing an anomaly detection model in the following ways:

(1): 

## Exploratory Data Analysis

### Cleaning the Data Set

### Transforming the data set

### Exporting Transformed Data Set to CSV

In [8]:
df.to_csv('../data/processed/nyc_taxi_transformed.csv')

### Visualizing the Data

First, some basic statistics of the data set and correlation coefficients are calculated. 

In [9]:
df.describe()

,value
count,10320.000000
mean,15137.569380
std,6939.495808
min,8.000000
25%,10262.000000
50%,16778.000000
75%,19838.750000
max,39197.000000


In [10]:
df.corr()

,value
value,1.0


###### Generating Interesting Plots

### Model Construction

### Testing Different Models

## Results